In [1]:
import duckdb
import pandas as pd

In [2]:
sql_query = '''
SELECT
    violation_code,
    SUM(fee_usd) AS total_revenue_usd
FROM
    silver_parking_violation_codes
GROUP BY
    violation_code
HAVING
    NOT(total_revenue_usd >= 1)
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,violation_code,total_revenue_usd
0,41,0.0
